In [11]:
#Готовые классы по расчету температур сред на выходе из охладителя дренажа
# температуре воды на выходе из СП1 и расходу пара в отбор СП1
# давлению в отборе на СП2 и расходу в отбор

import SP
import mat_properties as prop
import numpy as n
import pandas as pd  

#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

calctolerance=10**-3
calcmethod = "hybr"
KPD_SP=0.99 #эффективность сетевых подогревателей
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)


OD = SP.od('SP1-OD','OD-GPK','SWIN-OD','OD-SP1',    
                    KPD_SP,calctolerance,water,calcmethod,water_streams0,water_streams)
OD_res=OD.calc()    
SP1 = SP.sp1('OTB1-SP1','SP1-OD','WIN-SP1','SP1-SP2','SP2-SP1',    
                    KPD_SP,calctolerance,water,calcmethod,water_streams0,water_streams)
SP1_res=SP1.calc() 
SP2 = SP.sp2('OTB2-SP2','SP2-SP1','SP1-SP2','SP2-WOUT',    
                    KPD_SP,calctolerance,water,calcmethod,water_streams0,water_streams)
SP2_res=SP2.calc()
SP2_res

{'p_otb': 0.13219638630275696, 'Gotb': 43.9162988538204}

In [13]:
import numpy as n
import mat_properties as prop
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Достаю данные о теплообменниках из расчетного режима работы СП
Cp=4.187
#Psv=1 #давление сетевой воды
Qsp1_0=64.062*Cp/3.6 #нагрузка ПСГ-2 (расчетная точка)
Qsp2_0=82.397*Cp/3.6 #нагрузка ПСГ-1 (расчетная точка)
Qod_0=6.81*Cp/3.6 #нагрузка охладителя дренажа (расчетная точка)
t_0=-3.1 #температура расчетная наружного воздуха
tw2_0=98.56 #температура прямой сетевой воды (расчетная точка)
tw1_0=46 #температура обратной сетевой воды (расчетная точка)
Gsv_0=2886/3.6 #расход сетевой воды (расчетная точка)
KPD_SP=0.99 #эффективность сетевых подогревателей
Gsp2_0=157.44/3.6 #расход пара в ПСГ-2 (расчетная точка)
Gsp1_0=115.71/3.6 #расход пара в ПСГ-1 (расчетная точка)
tsp2_0=107.6 #температура пара в отборе на ПСГ-2 (расчетная точка)
tsp1_0=76 #температура пара в отборе на ПСГ-1 (расчетная точка)
ysp2_0=2.9/100 #доля влаги в  отборе на ПСГ-2 (расчетная точка)
ysp1_0=7.4/100 #доля влаги в  отборе на ПСГ-1 (расчетная точка)
dP=5/100 # потеря давления в паропроводах
Qw_0=Gsv_0*Cp*(tw2_0-tw1_0)/1000

Otb_sp2=water.t_q(tsp2_0,1-ysp2_0)
h_otbsp2_0=Otb_sp2['h']
p_otbsp2_0=Otb_sp2['P']
Otb_sp1_0=water.t_q(tsp1_0,1-ysp1_0)
h_otbsp1_0=Otb_sp1_0['h']
p_otbsp1_0=Otb_sp1_0['P']

Qsp2w_0=KPD_SP*Qsp2_0
Qsp1w_0=KPD_SP*(Qsp1_0)
Qodw_0=KPD_SP*Qod_0

p_sp2_0=(1-dP)*p_otbsp2_0
p_sp1_0=(1-dP)*p_otbsp1_0
Sp2_nas_0=water.p_q(p_sp2_0,0)
tsp2_0=Sp2_nas_0['T']
hsp2_nas_0=Sp2_nas_0['h']
Sp1_nas_0=water.p_q(p_sp1_0,0)
tsp1_0=Sp1_nas_0['T']
hsp1_nas_0=Sp1_nas_0['h']


tsp2_in_0=tw2_0-Qsp2w_0*1000/Gsv_0/Cp
tsp1_in_0=tw1_0+(Qodw_0)*1000/Gsv_0/Cp

deltasp1=tsp1_0-tsp2_in_0
deltasp2=tsp2_0-tw2_0
#Основная характеристика получена
KF_sp2=Gsv_0*Cp*n.log((tsp2_0-tsp2_in_0)/(tsp2_0-tw2_0))
KF_sp1=Gsv_0*Cp*n.log((tsp1_0-tsp1_in_0)/(tsp1_0-tsp2_in_0))

#Проверка
delt2=n.exp(-KF_sp2/Gsv_0/Cp)*(tsp2_0-tsp2_in_0)
delt1=n.exp(-KF_sp1/Gsv_0/Cp)*(tsp1_0-tsp1_in_0)
h_otbsp2_0
print(water)

In [3]:
#Расчет данных для охладителя дренажа
delta_0=10 #задаемся недогревом на холодном конце, чтобы оценить расход
tsp1_0 #температура горячего потока на входе 
G_dr_0=Gsp2_0+Gsp1_0 #расход греющего потока
tw1_0 #температура холодного потока на входе
tod_out_0=tsp1_0-delta_0 #температура холодного потока на выходе
tdr2_0=tsp1_0-Qod_0*1000/G_dr_0/Cp

deltat=((tsp1_0-tod_out_0)-(tdr2_0-tw1_0))/n.log((tsp1_0-tod_out_0)/(tdr2_0-tw1_0))
F=n.sqrt((tsp1_0-tdr2_0)*(tod_out_0-tw1_0))/deltat
#Соотношение расходов 


tsp1_0-tdr2_0

God_0=Qod_0*1000/Cp/(tod_out_0-tw1_0)
God_0/G_dr_0
#(tsp1_0-tdr2_0)/(tod_out_0-tw1_0)
dGod_0=God_0/Gsv_0 # доля расхода воды в ОД
dGod_0

tsp1_0
tdr2_0

49.8395111307518

In [4]:
#поверочный расчет ОД
Gdr=75.875
God=100
tdr_in=74.77
tw_in=46
F=F
Delta=tdr_in-tw_in
epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/F*n.sqrt(min(Gdr,God)/max(Gdr,God))))
epsilon
Q=epsilon*min(Gdr,God)*Cp*Delta/1000

tod_out=tw_in+Q*1000/God/Cp
tdr_out=tdr_in-Q*1000/Gdr/Cp
tdr_out

50.27861911361308

In [5]:
#делаю класс охладителя дренажа СП
import mat_properties as prop
import numpy as n

class OD:
    def __init__(self,stream11,stream12,stream21,stream22,KPD,calctolerance,water,calcmethod,water_streams0,water_streams):
        self.KPD=KPD
        self.water_streams0=water_streams0
        self.gas_streams=gas_streams
        self.water_streams=water_streams
        self.calcmethod=calcmethod
        self.calctolerance=calctolerance
        self.stream11=stream11
        self.stream21=stream21
        self.stream12=stream12
        self.stream22=stream22
        delta_0=10 #задаемся недогревом на холодном конце, чтобы оценить расход        
        self.Cp=4.187
        Qod_0=6.81*self.Cp/3.6 #нагрузка охладителя дренажа (расчетная точка)
        tdr_in_0 = water_streams0.at[self.stream11,'T'] #температура горячего потока на входе 
        G_dr_0=water_streams0.at[self.stream11,'G'] #расход греющего потока
        tw1_0= water_streams0.at[self.stream21,'T']#температура холодного потока на входе
        tod_out_0=tdr_in_0-delta_0 #температура холодного потока на выходе
        tdr2_0=tdr_in_0-Qod_0*1000/G_dr_0/self.Cp
        deltat=((tdr_in_0-tod_out_0)-(tdr2_0-tw1_0))/n.log((tdr_in_0-tod_out_0)/(tdr2_0-tw1_0))
        self.F=n.sqrt((tdr_in_0-tdr2_0)*(tod_out_0-tw1_0))/deltat
    def calc(self):
        Gdr = water_streams.at[self.stream11,'G']
        God = water_streams.at[self.stream21,'G']
        tdr_in = water_streams.at[self.stream11,'T']
        tw_in = water_streams.at[self.stream21,'T']
        Delta=tdr_in-tw_in
        epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))
        Q=epsilon*min(Gdr,God)*self.Cp*Delta/1000
        tod_out=tw_in+Q*1000/God/self.Cp
        tdr_out=tdr_in-Q*1000/Gdr/self.Cp
        print(God)
        return [tdr_out,tod_out]
    
import pandas as pd   
#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

calctolerance=10**-3
calcmethod = "hybr"
KPD_SP=0.99 #эффективность сетевых подогревателей
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)


OD = OD('SP1-OD','OD-GPK','SWIN-OD','OD-SP1',    
                    KPD_SP,calctolerance,water,calcmethod,water_streams0,water_streams)
PPND_res=OD.calc()    
PPND_res

100.776730953489


[50.21452868443271, 64.48786290685057]

In [6]:
#поверочный расчет СП2
dP=5/100 # потеря давления в паропроводах
Gsv=800
tw_in=70
tw_out=98.56
KF=6000
#p_otb_sp=
h_otb_sp=2622.54964
#p_sp1_0=(1-dP)*p_otb_sp

m=n.exp(-KF/Gsv/Cp)
print(m)
tsp=(tw_out-tw_in*m)/(1-m)
Nas_sp=water.t_q(tsp,0)
hsp_nas=Nas_sp['h']
p_sp=Nas_sp['P']
p_otb=p_sp/(1-dP)
Qsp=Gsv*Cp*(tw_out-tw_in)/KPD_SP/1000
Gotb=Qsp*1000/(h_otb_sp-hsp_nas)
Gotb

0.16675015615081362


44.21785046091928

In [7]:
#делаю класс СП2 (верхний отбор по пару, второй по воде)

import mat_properties as prop
import numpy as n

class SP2:
    def __init__(self,stream11,stream12,stream21,stream22,KPD,calctolerance,water,calcmethod,water_streams0,water_streams):
        self.KPD=KPD
        self.water_streams0=water_streams0
        self.gas_streams=gas_streams
        self.water_streams=water_streams
        self.calcmethod=calcmethod
        self.calctolerance=calctolerance
        self.stream11=stream11
        self.stream21=stream21
        self.stream12=stream12
        self.stream22=stream22
        self.Cp=4.187
        self.dP=5/100
        tsp2_0=water_streams0.at[self.stream12,'T'] #температура насыщения в номинале 
        tsp2_in_0 = water_streams0.at[self.stream21,'T'] #температура холодного поткоа на входе в номинале 
        tw2_0 = water_streams0.at[self.stream22,'T'] #температура воды на выходе
        Gsv_0 =water_streams0.at[self.stream21,'G'] #расход нагреваемого потока
        self.KF=Gsv_0*self.Cp*n.log((tsp2_0-tsp2_in_0)/(tsp2_0-tw2_0))

    def calc(self):
        Gsv=water_streams.at[self.stream21,'G'] #расход нагреваемого потока
        tw_out= water_streams.at[self.stream22,'T'] #температура воды на выходе
        tw_in = water_streams.at[self.stream21,'T'] #температура холодного поткоа на входе
        h_otb_sp= water_streams0.at[self.stream11,'H'] #температура воды на выходе
        m=n.exp(-self.KF/Gsv/self.Cp)
        
        tsp=(tw_out-tw_in*m)/(1-m)
        Nas_sp=water.t_q(tsp,0)
        hsp_nas=Nas_sp['h']
        p_sp=Nas_sp['P']
        p_otb=p_sp/(1-self.dP)
        Qsp=Gsv*self.Cp*(tw_out-tw_in)/KPD_SP/1000
        Gotb=Qsp*1000/(h_otb_sp-hsp_nas)    
        print(h_otb_sp)
        return [p_otb,Gotb]
    
import pandas as pd   
#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

calctolerance=10**-3
calcmethod = "hybr"
KPD_SP=0.99 #эффективность сетевых подогревателей
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

SP2 = SP2('OTB2-SP2','SP2-SP1','SP1-SP2','SP2-WOUT',    
                    KPD_SP,calctolerance,water,calcmethod,water_streams0,water_streams)
PPND_res=SP2.calc()    
PPND_res

2622.54964649625


[0.13219638630275696, 43.9162988538204]

In [8]:
#поверочный расчет СП1
dP=5/100 # потеря давления в паропроводах
Gsv=800
tw_in=48
tw_out
KF=6000
p_otb_sp=0.14
h_otb_sp=2700
p_sp1_0=(1-dP)*p_otb_sp
m=n.exp(-KF/Gsv/Cp)

In [9]:
#делаю класс СП1 (верхний отбор по пару, второй по воде)

import mat_properties as prop
import numpy as n

class SP1:
    def __init__(self,stream11,stream12,stream21,stream22,stream111, KPD,calctolerance,water,calcmethod,water_streams0,water_streams):
        self.KPD=KPD
        self.water_streams0=water_streams0
        self.gas_streams=gas_streams
        self.water_streams=water_streams
        self.calcmethod=calcmethod
        self.calctolerance=calctolerance
        self.stream11=stream11
        self.stream21=stream21
        self.stream12=stream12
        self.stream22=stream22
        self.stream111=stream111
        self.Cp=4.187
        self.dP=5/100
        tsp1_0=water_streams0.at[self.stream12,'T'] #температура насыщения в номинале 
        tsp1_in_0 = water_streams0.at[self.stream21,'T'] #температура холодного потока на входе в номинале 
        tw1_0 = water_streams0.at[self.stream22,'T'] #температура воды на выходе
        Gsv_0 =water_streams0.at[self.stream21,'G'] #расход нагреваемого потока
        self.KF=Gsv_0*self.Cp*n.log((tsp1_0-tsp1_in_0)/(tsp1_0-tw1_0))

    def calc(self):
        Gsv=water_streams.at[self.stream21,'G'] #расход нагреваемого потока
        tw_in = water_streams.at[self.stream21,'T'] #температура холодного поткоа на входе
        h_otb_sp= water_streams0.at[self.stream11,'H'] #температура воды на выходе
        p_otb_sp= water_streams0.at[self.stream11,'P'] #температура воды на выходе
        h_nas_sp2= water_streams0.at[self.stream111,'H'] #температура воды на выходе
        G_nas_sp2= water_streams0.at[self.stream111,'G'] #температура воды на выходе
        m=n.exp(-self.KF/Gsv/self.Cp)
        p_sp=p_otb_sp*(1-self.dP)
        Nas_sp=water.p_q(p_sp,0)
        t_sp=Nas_sp['T']
        hsp_nas=Nas_sp['h']
        tw_out=t_sp-(t_sp-tw_in)*m     
        Qsp=Gsv*self.Cp*(tw_out-tw_in)/KPD_SP/1000
        Gotb=(Qsp*1000-G_nas_sp2*(h_nas_sp2-hsp_nas))/(h_otb_sp-hsp_nas)    
        print(h_otb_sp)
        return [tw_out,Gotb]
    
import pandas as pd   
#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

calctolerance=10**-3
calcmethod = "hybr"
KPD_SP=0.99 #эффективность сетевых подогревателей
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

SP1 = SP1('OTB1-SP1','SP1-OD','WIN-SP1','SP1-SP2','SP2-SP1',    
                    KPD_SP,calctolerance,water,calcmethod,water_streams0,water_streams)
PPND_res=SP1.calc()    
PPND_res

2464.75601712861


[70.29564942694385, 35.523780484440394]